In [ ]:
%pip install finance-datareader

In [ ]:
import FinanceDataReader as fdr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


samsung = fdr.DataReader('005930', '2016')


print(samsung)

openValues = samsung[['Open']]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))
scaled = scaler.fit_transform(openValues)

TEST_SIZE = 200
train_data = scaled[:-TEST_SIZE]
test_data = scaled[-TEST_SIZE:]


def make_sample(data, window):
    train = []
    target = []
    for i in range(len(data)-window):
        train.append(data[i:i+window])
        target.append(data[i+window])
    return np.array(train), np.array(target)

X_train, y_train = make_sample(train_data, 30)

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN
import keras.losses as losses
import numpy as np


model = Sequential()

model.add(LSTM(16,
                input_shape=(X_train.shape[1], 1),
                activation='tanh',
                return_sequences=False)
          )
model.add(Dense(1))

#model.add(SimpleRNN(16, activation='tanh', input_shape=(X_train.shape[1], 1)))
#model.add(Dense(1))

#model.add(SimpleRNN(16, activation='tanh', return_sequences=True, input_shape=(X_train.shape[1], 1)))
#model.add(SimpleRNN(20))
#model.add(Dense(1))


model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(X_train, y_train, epochs = 100, batch_size = 16)

X_test, y_test = make_sample(test_data, 30)
pred = model.predict(X_test)

import matplotlib.pyplot as plt
plt.figure(figsize=(12, 9))
plt.plot(y_test*scaler.data_max_, label='stock price')
plt.plot(pred*scaler.data_max_, label='predicted stock price')
print('scale',scaler.data_max_)
pred_n=pred*scaler.data_max_
y_test_n=y_test*scaler.data_max_
print('# RMSE ',np.mean(losses.mean_squared_error(pred_n,y_test_n)**0.5))
print('# X_train y_train shape',X_train.shape,y_train.shape)
print('# X_test y_test shape',X_test.shape,y_test.shape)
plt.legend()
plt.show()